In [18]:
import pinecone

from google.cloud import bigquery

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import BigQueryLoader

from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("OPENAI_KEY")
pinecone_key = os.getenv('PINECONE_API_KEY')
pinecone_env = os.getenv('PINECONE_ENV')

In [19]:
pinecone.init(api_key=pinecone_key,
              environment=pinecone_env)

In [20]:
# one itinerary

In [21]:
 # Peru Panorama

In [22]:
# query = """
#     SELECT 
#         MAX(tour_operator) AS tour_operator,
#         tour_name,
#         itinerary_name,
#         MAX(visited_countries) AS visited_countries,
#         MAX(duration) AS duration,
#         MAX(Travel_Style) AS Travel_Style,
#         MAX(Service_Level) AS Service_Level,
#         MAX(Physical_Grading) AS Physical_Grading,
#         MAX(Merchandising) AS Merchandising,
#         MAX(Trip_Type) AS Trip_Type,
#         MAX(itinerary) AS itinerary,
#         MAX(url) AS url
#     FROM {PROJECT}.{DATASET}.{TABLE}
#     WHERE 
#       tour_name = 'Morocco Kasbahs & Desert'
#     GROUP BY
#       tour_name, itinerary_name;
#     """


In [23]:
# # Create index if it doesn't exist already
# if "chatbot" not in pinecone.list_indexes():
#     pinecone.create_index(
#         name="chatbot",
#         dimension=1536, 
#         metric="cosine" 
#     )

# # Load embeddings model  
# embeddings = OpenAIEmbeddings(openai_api_key=api_key)

# # Load texts
# loader = BigQueryLoader(query, metadata_columns=["tour_name"])

# docs = loader.load()

# text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# texts = text_splitter.split_documents(docs)



In [24]:
# # Create Pinecone index instance
# index = pinecone.Index("chatbot")

# # Upsert texts into index
# docsearch = Pinecone.from_documents(texts, embeddings, index_name="chatbot")

In [25]:
# 2 itienraries

In [26]:
# found_itineraries = ['Morocco Kasbahs & Desert', 'Coastal Morocco: Waves & Market Stalls']

In [27]:
# # Create index if it doesn't exist already
# if "chatbot" not in pinecone.list_indexes():
#     pinecone.create_index(
#         name="chatbot",
#         dimension=1536, 
#         metric="cosine" 
#     )

# for itinerary in found_itineraries:
#     query = f"""
#     SELECT 
#         MAX(tour_operator) AS tour_operator,
#         tour_name,
#         itinerary_name,
#         MAX(visited_countries) AS visited_countries,
#         MAX(duration) AS duration,
#         MAX(Travel_Style) AS Travel_Style,
#         MAX(Service_Level) AS Service_Level,
#         MAX(Physical_Grading) AS Physical_Grading,
#         MAX(Merchandising) AS Merchandising,
#         MAX(Trip_Type) AS Trip_Type,
#         MAX(itinerary) AS itinerary,
#         MAX(url) AS url
#     FROM {PROJECT}.{DATASET}.{TABLE}
#     WHERE 
#       tour_name = '{itinerary}'
#     GROUP BY
#       tour_name, itinerary_name;
#     """
    
#     # Load embeddings model  
#     embeddings = OpenAIEmbeddings(openai_api_key=api_key)

#     # Load texts
#     loader = BigQueryLoader(query, metadata_columns=["tour_name"])

#     docs = loader.load()

#     text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

#     texts = text_splitter.split_documents(docs)

#     # Create Pinecone index instance
#     index = pinecone.Index("chatbot")

#     # Upsert texts into index
#     docsearch = Pinecone.from_documents(texts, embeddings, index_name="chatbot")
#     print('Done')

In [28]:
# all itineraries

In [29]:

PROJECT = "wagon-bootcamp-377120"
DATASET = "g_adventures_dataset"
TABLE = "one_month"

query = f"""
    SELECT 
      DISTINCT tour_name
    FROM {PROJECT}.{DATASET}.{TABLE}
    """

client = bigquery.Client(project="wagon-bootcamp-377120")
query_job = client.query(query)
result = query_job.result()
df = result.to_dataframe()

In [30]:
tour_name_list = df['tour_name']

In [31]:
len(tour_name_list)

351

In [32]:
tour_name_list

0      Western and Central Europe: Venice, the Alps &...
1       Costa Rica: Volcanoes, Cloud Forests & Zip-Lines
2      Cancún Escape: Beaches, Cenotes, & Crystal-Cle...
3      Yucatan & Cancún: Merida, Mayan Ruins, & Beach...
4      Inca Adventure: Cusco, Machu Picchu & the Best...
                             ...                        
346                      Explore Kruger & Victoria Falls
347    Morocco Family Journey: Ancient Souks to the S...
348           Beijing to Tokyo: The Great Wall & Mt Fuji
349                                        Wellness Bali
350                                  Wellness Costa Rica
Name: tour_name, Length: 351, dtype: object

In [33]:
import time

# Create index if it doesn't exist already
if "chatbot" not in pinecone.list_indexes():
    pinecone.create_index(
        name="chatbot",
        dimension=1536, 
        metric="cosine" 
    )

for itinerary in tour_name_list:
    print(f'Started: {itinerary}')
    query = f"""
    SELECT 
        MAX(tour_operator) AS tour_operator,
        tour_name,
        itinerary_name,
        MAX(visited_countries) AS visited_countries,
        MAX(duration) AS duration,
        MAX(Travel_Style) AS Travel_Style,
        MAX(Service_Level) AS Service_Level,
        MAX(Physical_Grading) AS Physical_Grading,
        MAX(Merchandising) AS Merchandising,
        MAX(Trip_Type) AS Trip_Type,
        MAX(itinerary) AS itinerary,
        MAX(url) AS url
    FROM {PROJECT}.{DATASET}.{TABLE}
    WHERE 
      tour_name = "{itinerary}"
    GROUP BY
      tour_name, itinerary_name;
    """
    
    # Load embeddings model  
    embeddings = OpenAIEmbeddings(openai_api_key=api_key)

    # Load texts
    loader = BigQueryLoader(query, metadata_columns=["tour_name"])

    docs = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

    texts = text_splitter.split_documents(docs)

    # Create Pinecone index instance
    index = pinecone.Index("chatbot")

    # Upsert texts into index
    docsearch = Pinecone.from_documents(texts, embeddings, index_name="chatbot")
    time.sleep(1)
    print(f'Done: {itinerary}')

Started: Western and Central Europe: Venice, the Alps & the Flavours of Rome


Created a chunk of size 1490, which is longer than the specified 500
Created a chunk of size 573, which is longer than the specified 500
Created a chunk of size 587, which is longer than the specified 500
Created a chunk of size 1885, which is longer than the specified 500


Done: Western and Central Europe: Venice, the Alps & the Flavours of Rome
Started: Costa Rica: Volcanoes, Cloud Forests & Zip-Lines
Done: Costa Rica: Volcanoes, Cloud Forests & Zip-Lines
Started: Cancún Escape: Beaches, Cenotes, & Crystal-Clear Water
Done: Cancún Escape: Beaches, Cenotes, & Crystal-Clear Water
Started: Yucatan & Cancún: Merida, Mayan Ruins, & Beachside Livin'
Done: Yucatan & Cancún: Merida, Mayan Ruins, & Beachside Livin'
Started: Inca Adventure: Cusco, Machu Picchu & the Best Views Ever
Done: Inca Adventure: Cusco, Machu Picchu & the Best Views Ever
Started: Essential Western Europe: Amsterdam, Berlin & Epic Views


Created a chunk of size 1513, which is longer than the specified 500
Created a chunk of size 709, which is longer than the specified 500
Created a chunk of size 1943, which is longer than the specified 500
Created a chunk of size 527, which is longer than the specified 500
Created a chunk of size 814, which is longer than the specified 500
Created a chunk of size 751, which is longer than the specified 500


Done: Essential Western Europe: Amsterdam, Berlin & Epic Views
Started: Ultimate Vietnam: Big Cities, Beaches & The Best Views Ever
Done: Ultimate Vietnam: Big Cities, Beaches & The Best Views Ever
Started: Epic Vietnam: Hanoi, Ho Chi Minh City & Handmade Noodles
Done: Epic Vietnam: Hanoi, Ho Chi Minh City & Handmade Noodles
Started: North & Central Vietnam: Hanoi, Hoi An & Countryside Adventures
Done: North & Central Vietnam: Hanoi, Hoi An & Countryside Adventures
Started: Cambodia to Bangkok: Phnom Penh, Siem Reap & Khao San Road Hangouts
Done: Cambodia to Bangkok: Phnom Penh, Siem Reap & Khao San Road Hangouts
Started: Costa Rica: Beaches, Wildlife & Wild Times
Done: Costa Rica: Beaches, Wildlife & Wild Times
Started: Epic Costa Rica: Craters, Rainforests & Hanging-10
Done: Epic Costa Rica: Craters, Rainforests & Hanging-10
Started: Central Mexico & Yucatan: Mexico City, Oaxaca, Oh Yeah
Done: Central Mexico & Yucatan: Mexico City, Oaxaca, Oh Yeah
Started: The Total Peru Package: Cit

...

Done: Moroccan Desert Adventure: River Canyons & Camels
Started: Morocco Adventure: Culture & Colour


Created a chunk of size 810, which is longer than the specified 500


Done: Morocco Adventure: Culture & Colour
Started: Discover Sri Lanka
Done: Discover Sri Lanka
Started: Explore Jordan
Done: Explore Jordan
Started: Morocco: Sahara & Beyond
Done: Morocco: Sahara & Beyond
Started: Iconic Japan


Created a chunk of size 1588, which is longer than the specified 500


Done: Iconic Japan
Started: Discover Australia
Done: Discover Australia
Started: Thailand Journey
Done: Thailand Journey
Started: Explore India & Nepal
Done: Explore India & Nepal
Started: Explore Kruger National Park
Done: Explore Kruger National Park
Started: Discover India
Done: Discover India
Started: New Zealand Journey
Done: New Zealand Journey
Started: Machu Picchu and the Amazon
Done: Machu Picchu and the Amazon
Started: Inca Explorer
Done: Inca Explorer
Started: Natural Highlights of Costa Rica
Done: Natural Highlights of Costa Rica
Started: Iconic Peru
Done: Iconic Peru
Started: Safari in Kenya & Tanzania
Done: Safari in Kenya & Tanzania
Started: Discover Moorish Spain
Done: Discover Moorish Spain
Started: Cultural Wonders of Mexico
Done: Cultural Wonders of Mexico
Started: Iconic Australia and New Zealand
Done: Iconic Australia and New Zealand
Started: Paris & Normandy Highlights
Done: Paris & Normandy Highlights
Started: North India Highlights
Done: North India Highlights
S

In [ ]:
saa

In [ ]:
# ingestion complete - test Q&A

In [ ]:
pinecone_index = pinecone.Index("chatbot")
vectorstore = Pinecone(pinecone_index, embeddings.embed_query, "text")

In [ ]:
filters = {"tour_name": "Coastal Morocco: Waves & Market Stalls"}

# completion llm
llm = ChatOpenAI(
    openai_api_key=api_key,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'filter': filters})
)

In [ ]:
answer = qa.run('what surfing oppertunities are there?')
print(answer)

In [17]:
pinecone.delete_index("chatbot")